# Scribing Assignment - Part II

# Import libraries

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data

In [ ]:
## mount to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Load data

In [ ]:
# load text and covert to lowercase
filename = '/content/drive/MyDrive/Dataset/assignment4/testdata.txt'
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()  ## convert the text to lower case

In [ ]:
print(raw_text[:1000])

three rings for the elven-kings under the sky,
               seven for the dwarf-lords in their halls of stone,
            nine for mortal men doomed to die,
              one for the dark lord on his dark throne
           in the land of mordor where the shadows lie.
               one ring to rule them all, one ring to find them,
               one ring to bring them all and in the darkness bind them
           in the land of mordor where the shadows lie.
           
foreword

this tale grew in the telling, until it became a history of the great war of the ring and included many glimpses of the yet more ancient history that preceded it. it was begun soon after _the hobbit_ was written and before its publication in 1937; but i did not go on with this sequel, for i wished first to complete and set in order the mythology and legends of the elder days, which had then been taking shape for some years. i desired to do this for my own satisfaction, and i had little hope that other people 

## data preprocessing

In [ ]:
raw_text = raw_text.replace('\n\n','\n').replace('\n','|')  ## | is used to represent EOS --> END OF SENTENCE

In [ ]:
raw_text[:1000]

'three rings for the elven-kings under the sky,|               seven for the dwarf-lords in their halls of stone,|            nine for mortal men doomed to die,|              one for the dark lord on his dark throne|           in the land of mordor where the shadows lie.|               one ring to rule them all, one ring to find them,|               one ring to bring them all and in the darkness bind them|           in the land of mordor where the shadows lie.|           |foreword|this tale grew in the telling, until it became a history of the great war of the ring and included many glimpses of the yet more ancient history that preceded it. it was begun soon after _the hobbit_ was written and before its publication in 1937; but i did not go on with this sequel, for i wished first to complete and set in order the mythology and legends of the elder days, which had then been taking shape for some years. i desired to do this for my own satisfaction, and i had little hope that other people 

In [ ]:
## PUNCTUATION REMOVAL
import string

punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
punctuations = punctuations.replace('|','')  ## replace '|' from string of punctuations. we need '|' to represent EOS
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{}~


In [ ]:
def remove_punc(text):
    # Create a translation table to remove punctuation characters
    translator = str.maketrans('','', punctuations)

    # Use the translation table to remove punctuation
    text = text.translate(translator)
    return text

In [ ]:
raw_text = remove_punc(raw_text)

In [ ]:
raw_text[:1000]

'three rings for the elvenkings under the sky|               seven for the dwarflords in their halls of stone|            nine for mortal men doomed to die|              one for the dark lord on his dark throne|           in the land of mordor where the shadows lie|               one ring to rule them all one ring to find them|               one ring to bring them all and in the darkness bind them|           in the land of mordor where the shadows lie|           |foreword|this tale grew in the telling until it became a history of the great war of the ring and included many glimpses of the yet more ancient history that preceded it it was begun soon after the hobbit was written and before its publication in 1937 but i did not go on with this sequel for i wished first to complete and set in order the mythology and legends of the elder days which had then been taking shape for some years i desired to do this for my own satisfaction and i had little hope that other people would be intereste

### create mapping of unique chars to integers and integers to characters

In [ ]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i,c) for i,c in enumerate(chars))

In [ ]:
char_to_int

{' ': 0,
 '0': 1,
 '1': 2,
 '2': 3,
 '3': 4,
 '4': 5,
 '5': 6,
 '6': 7,
 '7': 8,
 '8': 9,
 '9': 10,
 'a': 11,
 'b': 12,
 'c': 13,
 'd': 14,
 'e': 15,
 'f': 16,
 'g': 17,
 'h': 18,
 'i': 19,
 'j': 20,
 'k': 21,
 'l': 22,
 'm': 23,
 'n': 24,
 'o': 25,
 'p': 26,
 'q': 27,
 'r': 28,
 's': 29,
 't': 30,
 'u': 31,
 'v': 32,
 'w': 33,
 'x': 34,
 'y': 35,
 'z': 36,
 '|': 37,
 '\x96': 38,
 'á': 39,
 'â': 40,
 'ä': 41,
 'é': 42,
 'ë': 43,
 'í': 44,
 'ó': 45,
 'ú': 46,
 'û': 47}

In [ ]:
int_to_char

{0: ' ',
 1: '0',
 2: '1',
 3: '2',
 4: '3',
 5: '4',
 6: '5',
 7: '6',
 8: '7',
 9: '8',
 10: '9',
 11: 'a',
 12: 'b',
 13: 'c',
 14: 'd',
 15: 'e',
 16: 'f',
 17: 'g',
 18: 'h',
 19: 'i',
 20: 'j',
 21: 'k',
 22: 'l',
 23: 'm',
 24: 'n',
 25: 'o',
 26: 'p',
 27: 'q',
 28: 'r',
 29: 's',
 30: 't',
 31: 'u',
 32: 'v',
 33: 'w',
 34: 'x',
 35: 'y',
 36: 'z',
 37: '|',
 38: '\x96',
 39: 'á',
 40: 'â',
 41: 'ä',
 42: 'é',
 43: 'ë',
 44: 'í',
 45: 'ó',
 46: 'ú',
 47: 'û'}

In [ ]:
chars

[' ',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|',
 '\x96',
 'á',
 'â',
 'ä',
 'é',
 'ë',
 'í',
 'ó',
 'ú',
 'û']

### summarize the loaded data

In [ ]:
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)

Total Characters:  982155
Total Vocab:  48


# prepare the dataset

In [ ]:
seq_length = 7  ## length of one sequence /timesteps
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)  ## number of patters / datapoints
print("Total Patterns: ", n_patterns)

Total Patterns:  982148


In [ ]:
## print a sample datapoint. You can uncomment to see the sample
print(dataX[0])
print(dataY[0])

[30, 18, 28, 15, 15, 0, 28]
19


In [ ]:
# reshape X to be [samples, time steps] --> embedding layer in LSTM accepts the inputs in this shape only.
# timesteps = sequence length
# convert dataX to torch tensor and reshaping
X = torch.tensor(dataX, dtype=torch.float32).reshape(n_patterns, seq_length, 1)
# X = X / float(n_vocab)
# convert dataY to torch tensor
y = torch.tensor(dataY, dtype=torch.int64)

In [ ]:
X

tensor([[[30.],
         [18.],
         [28.],
         ...,
         [15.],
         [ 0.],
         [28.]],

        [[18.],
         [28.],
         [15.],
         ...,
         [ 0.],
         [28.],
         [19.]],

        [[28.],
         [15.],
         [15.],
         ...,
         [28.],
         [19.],
         [24.]],

        ...,

        [[30.],
         [18.],
         [15.],
         ...,
         [21.],
         [19.],
         [24.]],

        [[18.],
         [15.],
         [ 0.],
         ...,
         [19.],
         [24.],
         [17.]],

        [[15.],
         [ 0.],
         [21.],
         ...,
         [24.],
         [17.],
         [37.]]])

In [ ]:
X.shape

torch.Size([982148, 7, 1])

In [ ]:
y

tensor([19, 24, 17,  ..., 17, 37, 37])

In [ ]:
y.shape

torch.Size([982148])

## Write code to prepare a train, val and test split from X and y to create

X_train, y_train

X_val, y_val

X_test, y_test

In [ ]:
from sklearn.model_selection import train_test_split

# Split the data into test (80%) and temporary (20%)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Further split the temporary data into training (80%) and validation (20%)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42)

# Print the sizes of the splits
print("Training set size:", len(X_train))
print("Validation set size:", len(X_val))
print("Test set size:", len(X_test))

Training set size: 589288
Validation set size: 196430
Test set size: 196430


# Create NN architecture

## Custom LSTM network

In [ ]:
"""
parameters of LSTM/RNN/GRU layer:

input_size – The number of expected features in the input x

hidden_size – The number of features in the hidden state h

num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two LSTMs together to form a stacked LSTM,
with the second LSTM taking in outputs of the first LSTM and computing the final results

batch_first – If True, then the input and output tensors are provided as (batch, seq, feature)

"""
######
# hyperparameters
#lstm_embeding_dim = ??
#lstm_hid_size = ??
#lstm_layers=  ??


######

class CharModel_LSTM(nn.Module):
    def __init__(self):
        super().__init__()
        #Create a suitable lstm network with an embedding layer, 2 or 3 lstm layers
        #a linear layer. You can have a dropout if required.
        self.lstm = nn.LSTM(input_size=1, hidden_size=256, num_layers=1, batch_first=True)
        self.dropout = nn.Dropout(0.2)
        self.linear = nn.Linear(256, n_vocab)
    def forward(self, x):
        #Write the forward pass for your network
        x, _ = self.lstm(x)
        x = x[:, -1,:]
        x = self.linear(self.dropout(x))
        return x

## Custom RNN Network

In [ ]:
"""
parameters of LSTM/RNN/GRU layer:

input_size – The number of expected features in the input x

hidden_size – The number of features in the hidden state h

num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two RNNs together to form a stacked RNN,
with the second RNN taking in outputs of the first RNN and computing the final results

batch_first – If True, then the input and output tensors are provided as (batch, seq, feature)

"""
######
# hyperparameters
#rnn_embeding_dim = ??
#rnn_hid_size = ??
#rnn_layers=  ??


######

class CharModel_RNN(nn.Module):
    def __init__(self):
        super().__init__()
        #Create a suitable rnn network with an embedding layer, 2 or 3 rnn layers
        #a linear layer. You can have a dropout if required.
    def forward(self, x):
        #Write the forward pass for your network
        return x

## Custom GRU Net

In [ ]:
"""
parameters of LSTM/RNN/GRU layer:

input_size – The number of expected features in the input x

hidden_size – The number of features in the hidden state h

num_layers – Number of recurrent layers. E.g., setting num_layers=2 would mean stacking two GRU together to form a stacked GRU,
with the second GRU taking in outputs of the first GRU and computing the final results

batch_first – If True, then the input and output tensors are provided as (batch, seq, feature)

"""
######
# hyperparameters
#gru_embeding_dim = ??
#gru_hid_size = ??
#gru_layers=  ??


######

class CharModel_GRU(nn.Module):
    def __init__(self):
        super().__init__()
        #Create a suitable gru network with an embedding layer, 2 or 3 gru layers
        #a linear layer. You can have a dropout if required.
    def forward(self, x):
        #Write the forward pass for your network
        return x

# Training

1. Write code to train the LSTM network and store the model as a checkpoint. Use validation data to tune your hyperparameters learning rate, batch size, num epochs, etc.

In [ ]:
model = CharModel_LSTM()
if torch.cuda.is_available:
  print('cuda available')

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

cuda available


CharModel_LSTM(
  (lstm): LSTM(1, 256, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (linear): Linear(in_features=256, out_features=48, bias=True)
)

In [ ]:
# Set up the parameters for training,
# tune the tunable parameters if necessary using val data
batch_size = 40
optimizer = optim.Adam(model.parameters())
loss_fn = nn.CrossEntropyLoss(reduction="sum")
train_loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size)
val_loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size)
test_loader = data.DataLoader(data.TensorDataset(X_train, y_train), shuffle=True, batch_size=batch_size)

In [ ]:
import os
n_epochs = 4
best_model = None
### ckpt_path =  # set up check point path
## If you have stored a checkpoint, In this way you can resume training model after the last training step by just loading it from the directory given.
# if os.path.isfile(ckpt_path):
#     model.load_state_dict(torch.load(ckpt_path)[0])
#     print('loading from model check point')

best_loss = np.inf
for epoch in range(n_epochs):
    model.train()
    #Write training code
    for X_batch, y_batch in train_loader:
        y_pred = model(X_batch.to(device))
        loss = loss_fn(y_pred.to(device), y_batch.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    # Validation
    # Writevalidation code and tune parameters if necessary
    #Include codes to print necessary logs
    model.eval()
    loss = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            y_pred = model(X_batch.to(device))
            loss += loss_fn(y_pred.to(device), y_batch.to(device))
        if loss < best_loss:
            best_loss = loss
            best_model = model.state_dict()
        print("Epoch %d: Cross-entropy: %.4f" % (epoch, loss))
torch.save([best_model, char_to_int], "single-char.pth")

Epoch 0: Cross-entropy: 1169528.6250
Epoch 1: Cross-entropy: 1069955.0000
Epoch 2: Cross-entropy: 1014340.9375
Epoch 3: Cross-entropy: 975519.1875
Epoch 4: Cross-entropy: 948310.5625
Epoch 5: Cross-entropy: 929056.5625
Epoch 6: Cross-entropy: 907094.1875
Epoch 7: Cross-entropy: 898178.5000
Epoch 8: Cross-entropy: 886649.0625
Epoch 9: Cross-entropy: 875138.4375
Epoch 10: Cross-entropy: 865574.8750
Epoch 11: Cross-entropy: 859753.1875
Epoch 12: Cross-entropy: 852201.0625
Epoch 13: Cross-entropy: 847829.8125
Epoch 14: Cross-entropy: 846016.8125
Epoch 15: Cross-entropy: 837068.1875
Epoch 16: Cross-entropy: 833059.5625
Epoch 17: Cross-entropy: 828265.6250
Epoch 18: Cross-entropy: 823955.5000
Epoch 19: Cross-entropy: 819510.4375


# Generating character using trained model

In [ ]:
best_model, char_to_int = torch.load("single-char.pth")  ## loading saved model
n_vocab = len(char_to_int)
int_to_char = dict((i, c) for c, i in char_to_int.items())
model.load_state_dict(best_model)

In [ ]:
X_test.size()
np.random.choice(len(X_test))
char_to_int['g']

In [ ]:
#Test your model's generative performance using the following testing code
# randomly generate a prompt
n = 10 ## number of characters to predict
rand_index = np.random.choice(len(X_test))
prompt = X_test[rand_index]
next_char_actual = y_test[rand_index]

#pattern = [char_to_int[c] for c in prompt]

#---------------------------------------------------------------------------#
#prediction

model.eval()
print('Prompt: "%s"' % prompt)
#print('Next chars (actual):',next_char_actual )
#print('----')
#print('prediction:\n')

In [ ]:
#Use your model to predict the next seq_length characters and check the performance
# CODE HERE
seq_length = 7
raw_text = open('/content/drive/MyDrive/ie643ss/Question2_dataset.txt', 'r', encoding='utf-8').read()
raw_text = raw_text.lower()
raw_text = remove_punc(raw_text)
start = np.random.randint(0, len(raw_text)-seq_length)
prompt = raw_text[start:start+seq_length]
pattern = [char_to_int[c] for c in prompt]
with torch.no_grad():
    for i in range(n):
        # format input array of int into PyTorch tensor
        x = np.reshape(pattern, (1, len(pattern), 1))
        x = torch.tensor(x, dtype=torch.float32)
        # generate logits as output from the model
        prediction = model(x.to(device))
        # convert logits into one character
        index = int(prediction.argmax())
        result = int_to_char[index]
        print(result, end="")
        # append the new character into the prompt for the next iteration
        pattern.append(index)
        pattern = pattern[1:]
print()
print("Done.")

In [ ]:
#Write code to evaluate your model's performance on the full test data
#Use character error rate on the test data


2. Using the template above, train the RNN network and store the model as a checkpoint. Use validation data to tune your hyperparameters learning rate, batch size, num epochs, etc. And test the models performance on an arbitrary test data point (or a set of test data points). Also evaluate the performance on full test data using character error rate.

3. Using the template above, train the GRU network and store the model as a checkpoint. Use validation data to tune your hyperparameters learning rate, batch size, num epochs, etc. And test the models performance on an arbitrary test data point (or a set of test data points). Also evaluate the performance on full test data using character error rate.